In [91]:
import os

import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from getpass import getpass

In [26]:
time_to = int(datetime.now().timestamp())
time_from = int((time_to - timedelta(days=120).total_seconds()))
time_from, time_to

(1701893577, 1712261577)

In [24]:
resp = requests.get('https://api.kraken.com/0/public/OHLC?pair=WIFUSD&interval=60&since=1701891170')

data = resp.json()['result']['WIFUSD']


In [27]:
pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'count']).tail()

,time,open,high,low,close,vwap,volume,count
715,1712246400,3.8263,3.8786,3.7837,3.8495,3.7964,17854.04157,70
716,1712250000,3.8165,3.9251,3.7979,3.9104,3.8438,25927.91534,99
717,1712253600,3.9109,3.9602,3.7277,3.7436,3.8499,58290.87371,263
718,1712257200,3.7274,3.7439,3.6621,3.6621,3.7086,70596.96143,286
719,1712260800,3.6608,3.6608,3.6255,3.6255,3.6384,6372.34409,35


In [35]:
def read_ohlcv_date(symbol: str) -> pd.DataFrame:
    time_to = int(datetime.now().timestamp())
    time_from = int((time_to - timedelta(days=120).total_seconds()))
    resp = requests.get(f'https://api.kraken.com/0/public/OHLC?pair={symbol.upper()}USD&interval=60&since={time_from}')
    data = resp.json()['result'][f'{symbol.upper()}USD']
    df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'count'])
    df.drop(columns=['vwap', 'count'], inplace=True)
    df.rename(columns={'time': 'date'}, inplace=True)
    df['date'] = pd.to_datetime(df['date'], unit='s')
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)
    df['volume'] = df['volume'].astype(float)
    df.set_index('date', inplace=True)
    df = df.iloc[:-1]
    return df

In [120]:
sol_df = read_ohlcv_date('WIF')
sol_df.tail()

,open,high,low,close,volume
date,,,,,
2024-04-05 06:00:00,3.3127,3.3795,3.2948,3.3072,38033.53780
2024-04-05 07:00:00,3.3016,3.3301,3.2625,3.3186,8075.32601
2024-04-05 08:00:00,3.3181,3.3655,3.2748,3.3146,68336.72455
2024-04-05 09:00:00,3.3132,3.3368,3.0560,3.2968,331925.55652
2024-04-05 10:00:00,3.2900,3.3043,3.2105,3.2574,22139.86649


In [121]:
sol_df.dtypes

open      float64
high      float64
low       float64
close     float64
volume    float64
dtype: object

In [122]:
sol_df.ta.macd(append=True)
sol_df.ta.rsi(append=True)
sol_df.ta.bbands(append=True)
sol_df.ta.obv(append=True)

sol_df.ta.sma(length=20, append=True)
sol_df.ta.ema(length=50, append=True)
sol_df.ta.stoch(append=True)
sol_df.ta.adx(append=True)

sol_df.ta.willr(append=True)
sol_df.ta.cmf(append=True)
sol_df.ta.psar(append=True)

,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2
date,,,,
2024-03-06 12:00:00,NaN,NaN,0.02,0
2024-03-06 13:00:00,1.851100,NaN,0.02,0
2024-03-06 14:00:00,1.851100,NaN,0.04,0
2024-03-06 15:00:00,1.867800,NaN,0.04,0
2024-03-06 16:00:00,1.883832,NaN,0.04,0
...,...,...,...,...
2024-04-05 06:00:00,NaN,3.500812,0.20,0
2024-04-05 07:00:00,NaN,3.445909,0.20,0
2024-04-05 08:00:00,NaN,3.401988,0.20,0


In [123]:
sol_df.tail()

,open,high,low,close,volume,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,BBL_5_2.0,...,STOCHd_14_3_3,ADX_14,DMP_14,DMN_14,WILLR_14,CMF_20,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2
date,,,,,,,,,,,,,,,,,,,,,
2024-04-05 06:00:00,3.3127,3.3795,3.2948,3.3072,38033.53780,-0.130216,-0.033011,-0.097205,30.671212,3.259350,...,8.264155,22.262396,15.940831,33.515304,-88.976700,-0.434457,NaN,3.500812,0.2,0
2024-04-05 07:00:00,3.3016,3.3301,3.2625,3.3186,8075.32601,-0.131755,-0.027640,-0.104115,31.714775,3.251737,...,10.194849,23.414064,15.212433,34.167162,-87.423355,-0.439614,NaN,3.445909,0.2,0
2024-04-05 08:00:00,3.3181,3.3655,3.2748,3.3146,68336.72455,-0.131779,-0.022131,-0.109648,31.535408,3.305833,...,11.459098,23.993215,16.687559,32.051020,-82.940495,-0.356624,NaN,3.401988,0.2,0
2024-04-05 09:00:00,3.3132,3.3368,3.0560,3.2968,331925.55652,-0.131715,-0.017654,-0.114061,30.703240,3.294184,...,16.284392,25.744987,13.831432,39.901677,-60.185185,-0.031468,NaN,3.366850,0.2,0
2024-04-05 10:00:00,3.2900,3.3043,3.2105,3.2574,22139.86649,-0.133308,-0.015397,-0.117911,28.886203,3.254828,...,22.635906,27.371633,13.029212,37.587389,-63.261583,-0.040345,NaN,3.365500,0.2,0


In [124]:
sol_df['OBV_in_million'] = sol_df['OBV']/1e7
sol_df['MACD_histogram_12_26_9'] = sol_df['MACDh_12_26_9']

In [125]:
last_day_summary = sol_df.iloc[-1][['close',
    'MACD_12_26_9','MACD_histogram_12_26_9', 'RSI_14', 'BBL_5_2.0', 'BBM_5_2.0', 'BBU_5_2.0','SMA_20', 'EMA_50','OBV_in_million', 'STOCHk_14_3_3', 
    'STOCHd_14_3_3', 'ADX_14',  'WILLR_14', 'CMF_20', 
    'PSARl_0.02_0.2', 'PSARs_0.02_0.2'
]]

print("Summary of Technical Indicators for the Last Day:")
print(last_day_summary)

Summary of Technical Indicators for the Last Day:
close                      3.257400
MACD_12_26_9              -0.133308
MACD_histogram_12_26_9    -0.015397
RSI_14                    28.886203
BBL_5_2.0                  3.254828
BBM_5_2.0                  3.298920
BBU_5_2.0                  3.343012
SMA_20                     3.501200
EMA_50                     3.649228
OBV_in_million             0.435720
STOCHk_14_3_3             31.204246
STOCHd_14_3_3             22.635906
ADX_14                    27.371633
WILLR_14                 -63.261583
CMF_20                    -0.040345
PSARl_0.02_0.2                  NaN
PSARs_0.02_0.2             3.365500
Name: 2024-04-05 10:00:00, dtype: float64


In [52]:
OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

In [55]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [132]:
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)
# llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [133]:
prompt = ChatPromptTemplate.from_template("""
Assume the role as a leading Technical Analysis (TA) expert in the stock market, \
a modern counterpart to Charles Dow, John Bollinger, and Alan Andrews. \
Your mastery encompasses both stock fundamentals and intricate technical indicators. \
You possess the ability to decode complex market dynamics, \
providing clear insights and recommendations backed by a thorough understanding of interrelated factors. \
Your expertise extends to practical tools like the pandas_ta module, \
allowing you to navigate data intricacies with ease. \
As a TA authority, your role is to decipher market trends, make informed predictions, and offer valuable perspectives.

given {symbol} TA data as below on the last trading day, what will be the next few days possible crypto price movement? 

Summary of Technical Indicators for the Last Day:
{last_day_summary}""".format(symbol="WIF", last_day_summary=last_day_summary))

In [134]:
chain = prompt | llm | StrOutputParser()

In [135]:
response = chain.invoke({})

In [136]:
print(response)

Analyzing the given technical indicators for the cryptocurrency WIF on the last trading day, we can derive insights into its potential price movement in the near future. Let's break down the key indicators:

1. **MACD (Moving Average Convergence Divergence)**: The MACD is at -0.133308 with a histogram value of -0.015397, indicating that the MACD line is below the signal line. This typically suggests bearish momentum. However, the small histogram value suggests that bearish momentum is weakening.

2. **RSI (Relative Strength Index)**: With an RSI of 28.886203, the asset is in the oversold territory. This condition often precedes a potential reversal or correction upwards as sellers may start to exhaust.

3. **Bollinger Bands (BB)**: The closing price is very close to the lower Bollinger Band (BBL), which could indicate that the asset is oversold. This condition, similar to the RSI, often leads to a potential price bounce back towards the middle band (BBM) or higher.

4. **SMA (Simple Mo